In [1]:
# 导入工具包
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import SimpleRNN, LSTM, GRU
from keras.optimizers import SGD, Nadam, Adam, RMSprop
from keras.callbacks import TensorBoard
from keras.utils import np_utils
import scipy.io 
import numpy as np

Using TensorFlow backend.


# 第一步：导入数据

In [2]:
data = scipy.io.loadmat('DATA\sp1s_aa_1000Hz.mat') # 'dict' object 
# data = scipy.io.loadmat('DATA\sp1s_aa.mat')
y_test = np.loadtxt('DATA\labels of the test set.txt',encoding="utf-8")

print("len(data):",len(data))
print(data)
print("len(y_test):",len(y_test))

len(data): 7
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Fri Nov  8 16:06:57 2002', '__version__': '1.0', '__globals__': [], 'clab': array([[array(['F3'], dtype='<U2'), array(['F1'], dtype='<U2'),
        array(['Fz'], dtype='<U2'), array(['F2'], dtype='<U2'),
        array(['F4'], dtype='<U2'), array(['FC5'], dtype='<U3'),
        array(['FC3'], dtype='<U3'), array(['FC1'], dtype='<U3'),
        array(['FCz'], dtype='<U3'), array(['FC2'], dtype='<U3'),
        array(['FC4'], dtype='<U3'), array(['FC6'], dtype='<U3'),
        array(['C5'], dtype='<U2'), array(['C3'], dtype='<U2'),
        array(['C1'], dtype='<U2'), array(['Cz'], dtype='<U2'),
        array(['C2'], dtype='<U2'), array(['C4'], dtype='<U2'),
        array(['C6'], dtype='<U2'), array(['CP5'], dtype='<U3'),
        array(['CP3'], dtype='<U3'), array(['CP1'], dtype='<U3'),
        array(['CPz'], dtype='<U3'), array(['CP2'], dtype='<U3'),
        array(['CP4'], dtype='<U3'), array(['CP6'], dtype='<U3'

# 第二步：预处理数据

In [3]:
"""
将训练数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
x_train = data['x_train'].reshape((316,500,28))
x_train /= 200
x_train = x_train.astype('float32')
"""
将测试数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
x_test = data['x_test'].reshape((100,500,28))
x_test /= 200
x_test = x_test.astype('float32')

"""
将标签数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
y_train = data['y_train'].reshape(316,1)
tmp_train = []
for i in y_train:
    if i == 1:
        tmp_train.append(1)
    elif i == 0:
        tmp_train.append(-1)
y_train = np.array(tmp_train)
y_train = np_utils.to_categorical(y_train, 2)
y_train = y_train.astype('float32')

y_test = y_test.reshape(100,1)
tmp_test = []
for i in y_test:
    if i == 1:
        tmp_test.append(1)
    elif i == 0:
        tmp_test.append(-1)       
y_test = np.array(tmp_test)
y_test = np_utils.to_categorical(y_test, 2)
y_test = y_test.astype('float32')

In [4]:
# 第三步：构建训练模型

In [5]:
model = Sequential()
model.add(LSTM(10, return_sequences = True, input_shape=(500, 28)))
model.add(LSTM(10, return_sequences = True))
model.add(LSTM(5))
model.add(Dense(2, activation = 'softmax'))

model.summary()

"""
优化器设置
学习率为0.001
"""
optim = Nadam(lr = 0.001)
# 设置损失函数为交叉熵损失函数
model.compile(loss = 'categorical_crossentropy', optimizer = optim, metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500, 10)           1560      
_________________________________________________________________
lstm_2 (LSTM)                (None, 500, 10)           840       
_________________________________________________________________
lstm_3 (LSTM)                (None, 5)                 320       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 12        
Total params: 2,732
Trainable params: 2,732
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 第四步：训练模型

In [7]:
"""

epochs设置为10
batch_size设置为20

"""

model.fit(x_train, y_train, epochs=15, batch_size=20)


Epoch 1/15
316/316 [==============================] - 7s 23ms/step - loss: 0.5554 - accuracy: 0.9937
Epoch 2/15
316/316 [==============================] - 7s 21ms/step - loss: 0.3210 - accuracy: 1.0000
Epoch 3/15
316/316 [==============================] - 7s 22ms/step - loss: 0.1597 - accuracy: 1.0000
Epoch 4/15
316/316 [==============================] - 7s 21ms/step - loss: 0.0928 - accuracy: 1.0000
Epoch 5/15
316/316 [==============================] - 7s 22ms/step - loss: 0.0630 - accuracy: 1.0000
Epoch 6/15
316/316 [==============================] - 7s 22ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 7/15
316/316 [==============================] - 7s 22ms/step - loss: 0.0381 - accuracy: 1.0000
Epoch 8/15
316/316 [==============================] - 7s 23ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 9/15
316/316 [==============================] - 7s 22ms/step - loss: 0.0263 - accuracy: 1.0000
Epoch 10/15
316/316 [==============================] - 7s 22ms/step - loss: 0.0224 - accur

# 第五步：计算最后得分和精度

In [8]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=1)
print('测试得分:', score)
print('测试精度:', acc)

100/100 [==============================] - 10s 104ms/step
测试得分: 0.011368723306804895
测试精度: 1.0
